In [1]:
#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
using System;
using System.Threading;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
//using BoSSS.XNSE_Solver;
using BoSSS.Solution.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Utils;
using System.IO;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using MomentFittingVariants = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants;
Init();

In [2]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "Popcorn2D" : buildname; //Popcorn2D  VanishingSphere2D
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "convergenceAggStokesSteadyFinal" : dbname; //Collison´2
string table_name = String.Concat(buildname, "_", dbname);

In [3]:
using BoSSS.Solution.Gnuplot;


In [4]:
//ExecutionQueues
var myBatch = ExecutionQueues[5];
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName,myBatch);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; 
myDB

{ Session Count = 75; Grid Count = 5; Path = \\dc3\userspace\toprak\BoSSS-ConvSteadyFinal\Popcorn2D_convergenceAggStokesSteadyFinal }

In [5]:
myDB.Sessions.Where(s => s.SuccessfulTermination == true)

#0: Popcorn2D_convergenceAggStokesSteadyFinal	SolverTrue_J65536_k3_Re1000_agg0_ti0.000atZ_r0.6_AMRFalse_LBFalse_W9.99_Stokes_dRdt0.00mumps_ct1	8/18/2024 5:40:32 PM	97537f4f...
#1: Popcorn2D_convergenceAggStokesSteadyFinal	SolverTrue_J65536_k2_Re1000_agg0_ti0.000atZ_r0.6_AMRFalse_LBFalse_W9.99_Stokes_dRdt0.00mumps_ct1	8/18/2024 5:40:16 PM	8312659c...
#2: Popcorn2D_convergenceAggStokesSteadyFinal	SolverTrue_J65536_k1_Re1000_agg0_ti0.000atZ_r0.6_AMRFalse_LBFalse_W9.99_Stokes_dRdt0.00mumps_ct1	8/18/2024 5:40:01 PM	bd8474e2...
#3: Popcorn2D_convergenceAggStokesSteadyFinal	SolverTrue_J16384_k3_Re1000_agg0.4_ti0.000atZ_r0.6_AMRFalse_LBFalse_W9.99_Stokes_dRdt0.00mumps_ct1	8/18/2024 5:35:46 PM	a61665fe...
#4: Popcorn2D_convergenceAggStokesSteadyFinal	SolverTrue_J16384_k3_Re1000_agg0.1_ti0.000atZ_r0.6_AMRFalse_LBFalse_W9.99_Stokes_dRdt0.00mumps_ct1	8/18/2024 5:35:01 PM	c9c88eb3...
#5: Popcorn2D_convergenceAggStokesSteadyFinal	SolverTrue_J16384_k3_Re1000_agg0.2_ti0.000atZ_r0.6_AMRFalse_LBFalse_W9

In [6]:
var refSessions = myDB.Sessions.Where(s => s.Timesteps.First().Grid.NumberOfCells > 60000);
refSessions

#0: Popcorn2D_convergenceAggStokesSteadyFinal	SolverTrue_J65536_k3_Re1000_agg0_ti0.000atZ_r0.6_AMRFalse_LBFalse_W9.99_Stokes_dRdt0.00mumps_ct1	8/18/2024 5:40:32 PM	97537f4f...
#1: Popcorn2D_convergenceAggStokesSteadyFinal	SolverTrue_J65536_k2_Re1000_agg0_ti0.000atZ_r0.6_AMRFalse_LBFalse_W9.99_Stokes_dRdt0.00mumps_ct1	8/18/2024 5:40:16 PM	8312659c...
#2: Popcorn2D_convergenceAggStokesSteadyFinal	SolverTrue_J65536_k1_Re1000_agg0_ti0.000atZ_r0.6_AMRFalse_LBFalse_W9.99_Stokes_dRdt0.00mumps_ct1	8/18/2024 5:40:01 PM	bd8474e2...


In [9]:
BoSSSshell.WorkflowMgm.hConvergence.Update();

In [10]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns( "AgglomerationThreshold", "DGdegree:Velocity*","Grid:hMin","Grid:NoOfCells" ,"L2Error_"+ "VelocityX", "L2Error_" +"Pressure");
Tab2.Print();
Tab2.SaveToFile(table_name + $"All_at{timeStep}_summary.json");

    AgglomerationThreshold DGdegree:Velocity* Grid:hMin Grid:NoOfCells L2Error_VelocityX     L2Error_Pressure      
0:  0                      3                  0.0078125 65536          0                     0                     
1:  0                      2                  0.0078125 65536          0                     0                     
2:  0                      1                  0.0078125 65536          0                     0                     
3:  0.4                    3                  0.015625  16384          0                     0                     
4:  0.1                    3                  0.015625  16384          0                     0                     
5:  0.2                    3                  0.015625  16384          0                     0                     
6:  0.5                    3                  0.015625  16384          0                     0                     
7:  0.3                    3                  0.015625  16384          0

In [11]:
var grids = myDB.Sessions.Select(s => s.Timesteps.First().Grid.NumberOfCells);
int NCellsFinestMesh =  grids.Max(); //  57600; 
Math.Sqrt(NCellsFinestMesh)

256

In [12]:
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> 
                Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != NCellsFinestMesh       
                 );

In [14]:
using BoSSS.Solution.Gnuplot;
using System.Linq;

// arrange line color and types
var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
var allPointTypes = Enum.GetValues(typeof(PointTypes)).Cast<PointTypes>().ToArray();
PointTypes[] myPointTypes = new PointTypes[]{ PointTypes.Diamond, PointTypes.Box,  PointTypes.LowerTriangle,PointTypes.OpenLowerTriangle };


In [1]:
string xVarName = "Grid:hMin";
string yVarName = "L2error_VelocityX";
int dgDegree = 1;

// Filter rows depending on selection
var Tab3 = Tab2.ExtractRows((iRow,RowEntries)=> 
                //Convert.ToDouble(RowEntries["AgglomerationThreshold"]) == 0 &&
                Convert.ToDouble(RowEntries["DGdegree:Velocity*"]) == dgDegree
            );

// Sort in ascending order of AgglomerationThreshold
System.Data.DataView view = Tab3.DefaultView;
view.Sort = "AgglomerationThreshold ASC"; 
Tab3 = view.ToTable();

Tab3.Print()


Error: (6,5): error CS0246: The type or namespace name 'Plot2Ddata' could not be found (are you missing a using directive or an assembly reference?)
(6,35): error CS0246: The type or namespace name 'Plot2Ddata' could not be found (are you missing a using directive or an assembly reference?)
(10,51): error CS0246: The type or namespace name 'LineColors' could not be found (are you missing a using directive or an assembly reference?)
(10,69): error CS0246: The type or namespace name 'LineColors' could not be found (are you missing a using directive or an assembly reference?)
(11,55): error CS0246: The type or namespace name 'PointTypes' could not be found (are you missing a using directive or an assembly reference?)
(11,73): error CS0246: The type or namespace name 'PointTypes' could not be found (are you missing a using directive or an assembly reference?)
(12,13): error CS0246: The type or namespace name 'PointTypes' could not be found (are you missing a using directive or an ass

In [ ]:
Plot2Ddata plot = new Plot2Ddata();

In [ ]:
var plot = Tab3.ToPlot(xVarName, yVarName,  "AgglomerationThreshold");
plot.LogY = true;
plot.LogX = true;
        for (int i = 0; i < plot.dataGroups.Length; i++){
            var dataGroup = plot.dataGroups[i];
            dataGroup.Format.LineColor = allColors[i % allColors.Length];
            dataGroup.Format.PointType = myPointTypes[i % myPointTypes.Length];
        }   
plot.PlotNow()

In [ ]:
plot.SaveToGIF(yVarName + "_k" + dgDegree + "_2D.png");
plot.SavePgfplotsFile(yVarName + "_k" + dgDegree + "_2D.tex");

In [ ]:
plot.Regression()

In [20]:
PlotTable.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


Gnuplot Error: 
multiplot> plot "C:/Users/toprak/AppData/Local/Temp/ca6677f1-e2b4-4886-bcf5-377f383a3c1d" title "DGdegree:Velocity*3" with linespoints linecolor  "red" dashtype 1 linewidth 1 pointtype 13 pointsize 0.7, "C:/Users/toprak/AppData/Local/Temp/61c51368-eab6-4ee6-8974-81440f28c67d" title "DGdegree:Velocity*2" with linespoints linecolor  "green" dashtype 1 linewidth 1 pointtype 5 pointsize 0.7, "C:/Users/toprak/AppData/Local/Temp/fe7c81a8-b32f-44b6-a98e-d0962154f86a" title "DGdegree:Velocity*1" with linespoints linecolor  "blue" dashtype 1 linewidth 1 pointtype 9 pointsize 0.7
                                                                                                                                                                                                                                                                                                                                                                                                                       

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 VelocityX 
 
 
 DGdegree:Velocity*3 
 
 
 DGdegree:Velocity*3 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*2 
 
 
 DGdegree:Velocity*2 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*1 
 
 
 DGdegree:Velocity*1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 VelocityY 
 
 
 DGdegree:Velocity*3 
 
 
 DGdegree:Velocity*3 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*2 
 
 
 DGdegree:Velocity*2 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*1 
 
 
 DGdegree:Velocity*1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 Pressure 
 
 
 DGdegree:Pressure2 
 
 
 DGdegree:Pressure2 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Pressure1 
 
 
 DGdegree:Pressure1 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Pressure0 
 
 
 DGdegree:Pressure0

In [21]:
int cnt = 0;
foreach(var reg in RegressionS){
    Console.WriteLine(varNames[cnt]);
    var regB = reg.ToArray();
    for(int j = 0; j < regB.Length; j++){       
        int dgDeg = ( regB[j].Key).Last() - '0';
        Console.Write("DG degree: "+ dgDeg+ " \t");
        double slope = regB[j].Value;
        Console.Write("Slope: " +slope + "\t");

        if((slope - (dgDeg + 1) ) >= -0.2)
        Console.WriteLine("(+)"); //Console.WriteLine("Expected slope archieved");
        else
        Console.WriteLine("(-)");
    }
    cnt++;
}

VelocityX
DG degree: 3 	Slope: 3.491142290094649	(-)
DG degree: 2 	Slope: 2.5884183294619043	(-)
DG degree: 1 	Slope: 1.4746546665817304	(-)
VelocityY
DG degree: 3 	Slope: 3.5066102548935403	(-)
DG degree: 2 	Slope: 2.5747397545959267	(-)
DG degree: 1 	Slope: 1.4715735577547855	(-)
Pressure
DG degree: 2 	Slope: 2.7064738107474584	(-)
DG degree: 1 	Slope: 2.015717925321525	(+)
DG degree: 0 	Slope: 1.2708900054000984	(+)
Residual-ContiEq
DG degree: 3 	Slope: NaN	(-)
DG degree: 2 	Slope: NaN	(-)
DG degree: 1 	Slope: NaN	(-)
